# 2D方腔流测试代码

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# 设置随机种子
torch.manual_seed(1234)
np.random.seed(1234)

# 设备选择
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 神经网络定义
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 50),  # 输入：x,y,t
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 3)   # 输出：u,v,p
        )

    def forward(self, x, y, t):
        inputs = torch.cat([x, y, t], dim=1)
        return self.net(inputs)

In [ ]:
# PDE残差计算
def compute_pde_residual(model, x, y, t, nu=0.01, rho=1.0):
    inputs = torch.cat([x, y, t], dim=1)
    outputs = model(x, y, t)
    u, v, p = outputs[:, 0:1], outputs[:, 1:2], outputs[:, 2:3]

    # 自动微分
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]

    v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]

    p_x = torch.autograd.grad(p, x, grad_outputs=torch.ones_like(p), create_graph=True)[0]
    p_y = torch.autograd.grad(p, y, grad_outputs=torch.ones_like(p), create_graph=True)[0]

    # Navier-Stokes残差
    f_u = u_t + (u * u_x + v * u_y) + (1/rho) * p_x - nu * (u_xx + u_yy)
    f_v = v_t + (u * v_x + v * v_y) + (1/rho) * p_y - nu * (v_xx + v_yy)
    f_cont = u_x + v_y  # 连续性方程

    return f_u, f_v, f_cont

In [ ]:
# 训练函数
# epochs: 训练轮数 (10000), data_tensor: 域内数据, ic_tensor: 初始条件数据, bc_tensor: 边界条件数据, col_tensor: PDE残差点数据
def train_pinn(model, data_tensor, ic_tensor, bc_tensor, col_tensor, epochs=10000):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(epochs):
        optimizer.zero_grad()

        # 数据损失
        if data_tensor.shape[0] > 0:
            x_data, y_data, t_data, u_data, v_data, p_data = data_tensor[:, 0:1], data_tensor[:, 1:2], data_tensor[:, 2:3], data_tensor[:, 3:4], data_tensor[:, 4:5], data_tensor[:, 5:6]
            outputs = model(x_data, y_data, t_data)
            loss_data = torch.mean((outputs[:, 0:1] - u_data)**2 + (outputs[:, 1:2] - v_data)**2 + (outputs[:, 2:3] - p_data)**2)
        else:
            loss_data = 0.0

        # 初始条件损失
        x_ic, y_ic, t_ic, u_ic, v_ic, p_ic = ic_tensor[:, 0:1], ic_tensor[:, 1:2], ic_tensor[:, 2:3], ic_tensor[:, 3:4], ic_tensor[:, 4:5], ic_tensor[:, 5:6]
        outputs_ic = model(x_ic, y_ic, t_ic)
        loss_ic = torch.mean((outputs_ic[:, 0:1] - u_ic)**2 + (outputs_ic[:, 1:2] - v_ic)**2 + (outputs_ic[:, 2:3] - p_ic)**2)

        # 边界条件损失
        x_bc, y_bc, t_bc, u_bc, v_bc = bc_tensor[:, 0:1], bc_tensor[:, 1:2], bc_tensor[:, 2:3], bc_tensor[:, 3:4], bc_tensor[:, 4:5]
        outputs_bc = model(x_bc, y_bc, t_bc)
        loss_bc = torch.mean((outputs_bc[:, 0:1] - u_bc)**2 + (outputs_bc[:, 1:2] - v_bc)**2)

        # PDE损失
        x_col, y_col, t_col = col_tensor[:, 0:1], col_tensor[:, 1:2], col_tensor[:, 2:3]
        f_u, f_v, f_cont = compute_pde_residual(model, x_col, y_col, t_col) # type: ignore
        loss_pde = torch.mean(f_u**2 + f_v**2 + f_cont**2)

        # 总损失
        loss = loss_data + loss_ic + loss_bc + loss_pde
        loss.backward()
        optimizer.step()

        if epoch % 1000 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

In [ ]:

# 数据准备
def prepare_data():
    # 假设从老师处获取的CFD数据（示例：随机生成，实际应替换为您的CSV数据）
    N_data = 1000
    data = np.random.rand(N_data, 6)  # [x, y, t, u, v, p]
    data[:, 0] = data[:, 0]  # x in [0,1]
    data[:, 1] = data[:, 1]  # y in [0,1]
    data[:, 2] = data[:, 2] * 0.1  # t in [0,0.1]
    # 实际数据：替换为np.loadtxt('your_data.csv', delimiter=',') 或类似

    # 初始条件 (t=0)
    N_ic = 200
    x_ic = np.random.rand(N_ic)  # x in [0,1]
    y_ic = np.random.rand(N_ic)  # y in [0,1]
    t_ic = np.zeros(N_ic)        # t=0
    u_ic = np.zeros(N_ic)        # u=0 at t=0
    v_ic = np.zeros(N_ic)        # v=0 at t=0
    p_ic = np.random.rand(N_ic)  # 压力

    # 边界条件 (方腔流：上壁u=1, v=0；其他壁u=v=0)
    N_bc = 400
    x_bc = np.concatenate([np.random.rand(N_bc//4), np.zeros(N_bc//4), np.ones(N_bc//4), np.random.rand(N_bc//4)])
    y_bc = np.concatenate([np.ones(N_bc//4), np.random.rand(N_bc//4), np.random.rand(N_bc//4), np.zeros(N_bc//4)])
    t_bc = np.random.rand(N_bc) * 0.1
    u_bc = np.concatenate([np.ones(N_bc//4), np.zeros(N_bc//4), np.zeros(N_bc//4), np.zeros(N_bc//4)])  # 上壁u=1
    v_bc = np.zeros(N_bc)  # v=0

    # 域内collocation points（用于PDE残差）
    N_col = 10000
    x_col = np.random.rand(N_col)
    y_col = np.random.rand(N_col)
    t_col = np.random.rand(N_col) * 0.1

    # 转换为Tensor
    data_tensor = torch.tensor(data, dtype=torch.float32, requires_grad=True).to(device)
    ic_tensor = torch.tensor(np.vstack([x_ic, y_ic, t_ic, u_ic, v_ic, p_ic]).T, dtype=torch.float32, requires_grad=True).to(device)
    bc_tensor = torch.tensor(np.vstack([x_bc, y_bc, t_bc, u_bc, v_bc]).T, dtype=torch.float32, requires_grad=True).to(device)
    col_tensor = torch.tensor(np.vstack([x_col, y_col, t_col]).T, dtype=torch.float32, requires_grad=True).to(device)

    return data_tensor, ic_tensor, bc_tensor, col_tensor

In [5]:
# 主程序
model = PINN().to(device)
data_tensor, ic_tensor, bc_tensor, col_tensor = prepare_data()
train_pinn(model, data_tensor, ic_tensor, bc_tensor, col_tensor)

Epoch 0, Loss: 2.394162
Epoch 1000, Loss: 0.655213
Epoch 2000, Loss: 0.639555
Epoch 3000, Loss: 0.635849
Epoch 4000, Loss: 0.633574
Epoch 5000, Loss: 0.631993
Epoch 6000, Loss: 0.632088
Epoch 7000, Loss: 0.630527
Epoch 8000, Loss: 0.629127
Epoch 9000, Loss: 0.626241
